# Throw 40k jobs at a neural network
- For Gensim word2vec tutorial, see [here](https://rare-technologies.com/word2vec-tutorial/)
- Can add additional sentences in later training stages

In [1]:
import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle
import gensim

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import LsiModel

from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel

from sklearn.manifold import TSNE
import pandas as pd

/usr/local/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
import spacy
nlp = spacy.load('en')
from spacy.lang.en.stop_words import STOP_WORDS

/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/usr/local/lib/python3.6/site-packages/msgpack_numpy

/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/usr/local/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationW

In [5]:
from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

trigram_sentences = LineSentence('../models_data_lower/spacy_trigram_sentences_PARSED.txt')
word2vec_filepath = '../models_data_lower/word2vec_model_all'

In [8]:
%%time

# initiate the model and perform the first epoch of training
skill2vec = Word2Vec(trigram_sentences, size=100, window=5,
                    min_count=20, sg=1, workers=15, iter=15)

skill2vec.save(word2vec_filepath)
print("Done generating model.")

# perform 10 epochs of training
skill2vec.train(trigram_sentences,total_examples=skill2vec.corpus_count, epochs=skill2vec.iter)
skill2vec.save(word2vec_filepath)

Done generating model.
CPU times: user 49min 42s, sys: 15 s, total: 49min 57s
Wall time: 7min 33s


In [9]:
# load the finished model from disk
skill2vec = Word2Vec.load(word2vec_filepath)
skill2vec.init_sims()
print(u'Model loaded with {} training epochs.'.format(skill2vec.train_count))

Model loaded with 2 training epochs.


In [10]:
print(u'{:,} terms in the skill2vec vocabulary.'.format(len(skill2vec.wv.vocab)))


37,815 terms in the skill2vec vocabulary.


In [11]:
def get_related_terms(token, topn=20):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in skill2vec.most_similar(positive=[token], topn=topn):

        print(u'{:20} {}'.format(word, round(similarity, 3)))

In [13]:
get_related_terms(u'sql')

relational_databases 0.837
sql,                 0.836
pl/sql               0.833
t-sql                0.799
sql.                 0.795
ms_sql               0.786
mysql                0.782
sql_server           0.777
microsoft_sql_server 0.763
ms_sql_server        0.758
sql_query            0.755
relational_database  0.75
ssis                 0.747
relational           0.745
rdbms                0.741
sql,_pl/sql          0.736
stored_procedure     0.733
postgresql           0.729
ssis,_ssrs,          0.727
stored_procedures    0.725


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [9]:
get_related_terms(u'OOP')

design_patterns      0.753
object_oriented      0.742
MVC                  0.736
object-oriented      0.733
SOLID_principles     0.727
design_patterns,     0.71
Design_Patterns      0.697
JavaScript           0.689
functional_programming 0.688
OO_programming       0.687
object-oriented_design 0.686
Javascript           0.684
PHP                  0.678
JavaScript_frameworks 0.677
Java                 0.676
object-oriented_programming 0.675
Object_Oriented_Programming 0.666
design_patterns.     0.66
OO                   0.655
TDD,                 0.655


/home/goodgame/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [10]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = skill2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

In [11]:
word_algebra(add=[u'research', u'big_data'])

data_science


/home/goodgame/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [12]:
word_algebra(add=[u'OOP', u'Javascript'], 
             subtract=[u'PHP'])

design_patterns


/home/goodgame/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [30]:
word_algebra(add=[u'R', u'Python'])

Scala


/home/goodgame/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [14]:
word_algebra(subtract=[u'statistics', u'software_engineer'])

Balance


/home/goodgame/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


# Visualize

In [15]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in skill2vec.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda item: -item[2])

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the skill2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(skill2vec.wv.syn0norm[term_indices, :],
                            index=ordered_terms)

/home/goodgame/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  


In [16]:
tsne_input = word_vectors.drop(STOP_WORDS, errors=u'ignore')
tsne_input = tsne_input.head(5000)

In [17]:
tsne_input.tail()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
consisting,0.101075,-0.124671,-0.053412,-0.002653,0.052885,0.026908,0.029805,0.106990,-0.034660,0.010739,...,0.132688,0.014966,-0.151445,-0.097137,-0.035891,-0.029848,-0.029494,-0.255880,0.143620,-0.047623
Commitment,-0.033439,0.027899,0.042494,-0.028532,0.022058,-0.129767,0.105768,-0.003284,-0.022362,-0.015190,...,0.042056,0.025279,-0.067227,0.359786,-0.029906,-0.050543,0.023643,0.015070,0.075948,-0.055169
"availability,",-0.038573,0.128042,-0.264992,-0.078605,0.075379,0.039059,0.045143,0.007505,0.011616,-0.114580,...,0.153414,-0.138115,0.006792,0.206330,-0.106295,-0.072303,-0.048766,-0.079881,0.086225,0.010151
"proposals,",-0.108023,-0.218689,-0.110374,0.015023,0.059096,0.148794,0.030229,-0.087591,0.031594,-0.019095,...,0.055663,0.011706,-0.101290,0.093746,0.068940,0.073718,-0.055769,0.120279,0.004458,0.079589
Data_Analysis,-0.086461,-0.069959,-0.155552,-0.168851,-0.008061,-0.022595,0.065511,0.212154,0.035395,-0.023373,...,0.031608,0.038580,0.141056,0.162552,-0.029006,0.202396,0.021122,0.084000,-0.124686,-0.067988


In [18]:
tsne_filepath = u'tsne_model'
tsne_vectors_filepath = u'tsne_vectors.npy'

In [97]:
# %%time

# tsne = TSNE()
# tsne_vectors = tsne.fit_transform(tsne_input.values)

# with open(tsne_filepath, 'wb') as f:
#     pickle.dump(tsne, f)

# pd.np.save(tsne_vectors_filepath, tsne_vectors)

CPU times: user 1min 44s, sys: 9.42 s, total: 1min 54s
Wall time: 1min 54s


In [19]:
with open(tsne_filepath, 'rb') as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=[u'x_coord', u'y_coord'])

In [20]:
tsne_vectors[u'word'] = tsne_vectors.index

In [23]:
from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()
# output_file("tsne_bokeh.html")

Loading BokehJS ...

In [24]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title=u't-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= (u'pan, wheel_zoom, box_zoom,'
                           u'box_select, reset'),
                   active_scroll=u'wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = u'@word') )

# draw the words as circles on the plot
tsne_plot.circle(u'x_coord', u'y_coord', source=plot_data,
                 color=u'blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color=u'black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value(u'16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);

# Assess Model

In [48]:
def model_accuracy(model, questions):

    accuracy = model.wv.accuracy(questions)
    
    sum_corr = len(accuracy[-1]['correct'])
    sum_incorr = len(accuracy[-1]['incorrect'])
    total = sum_corr + sum_incorr
    percent = lambda a: a / total * 100
    
    print('Total tasks: {}, Correct: {:.2f}%, Incorrect: {:.2f}%'.format(total, percent(sum_corr), percent(sum_incorr)))

In [49]:
model_accuracy(skill2vec, 'question-words.txt')

Total tasks: 1954, Correct: 18.17%, Incorrect: 81.83%


In [47]:
skill2vec.wv.accuracy('question-words.txt')

[{'correct': [], 'incorrect': [], 'section': 'capital-common-countries'},
 {'correct': [], 'incorrect': [], 'section': 'capital-world'},
 {'correct': [], 'incorrect': [], 'section': 'currency'},
 {'correct': [('HOUSTON', 'TEXAS', 'ATLANTA', 'GEORGIA'),
   ('PHILADELPHIA', 'PENNSYLVANIA', 'MIAMI', 'FLORIDA'),
   ('PHILADELPHIA', 'PENNSYLVANIA', 'TAMPA', 'FLORIDA'),
   ('DALLAS', 'TEXAS', 'ATLANTA', 'GEORGIA'),
   ('AUSTIN', 'TEXAS', 'MIAMI', 'FLORIDA'),
   ('AUSTIN', 'TEXAS', 'ORLANDO', 'FLORIDA'),
   ('PORTLAND', 'OREGON', 'MIAMI', 'FLORIDA'),
   ('PORTLAND', 'OREGON', 'ORLANDO', 'FLORIDA'),
   ('ORLANDO', 'FLORIDA', 'DALLAS', 'TEXAS'),
   ('ORLANDO', 'FLORIDA', 'DENVER', 'COLORADO'),
   ('IRVINE', 'CALIFORNIA', 'HOUSTON', 'TEXAS'),
   ('IRVINE', 'CALIFORNIA', 'DALLAS', 'TEXAS')],
  'incorrect': [('HOUSTON', 'TEXAS', 'PHILADELPHIA', 'PENNSYLVANIA'),
   ('HOUSTON', 'TEXAS', 'PHOENIX', 'ARIZONA'),
   ('HOUSTON', 'TEXAS', 'BOSTON', 'MASSACHUSETTS'),
   ('HOUSTON', 'TEXAS', 'SEATTLE', 'WAS

## Additional Training

Possibly find a way to include the U.S. Dept. of Education's O\*NET Database:

https://www.onetonline.org/

```
model = gensim.models.Word2Vec.load('/tmp/mymodel')
model.train(more_sentences)
```

Update: can't just do online training to modify the Google pre-trained model, because I don't have the necessary vocabulary. So one option might be to train on all of Wikipedia (perhaps see [Google's documentation](https://code.google.com/archive/p/word2vec/) and a [pre-processing script they mention]().
